In [8]:
import pandas as pd
import json
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import shapely
import rasterio
import json
import pyproj
import numpy as np
from pathlib import Path
import multiprocessing

root = "/scistor/ivm/project/NSO/timeseries_50_baseline"
big_tiles_folder = root + "/big_tiles"
small_tiles_folder = root + "/small_tiles"
annotation_path = root + "/annotations_shp/fixed_substations_NL_annotations.shp"
json_folder = root + "/geojsons"

In [9]:
def split_geotiff(rasters_path, output_dir, width, height, pixel_size):
    """
    This function splits a given geotiff raster image into multiple tiles with a specified width and height.

    Parameters:
    rasters_path (str): File path to the input folder with the rasters.
    output_dir (str): File path to the output folder where the created tiles will be saved.
    width (int): Width of the tiles in pixels.
    height (int): Height of the tiles in pixels.
    pixel_size (float): Pixel size of the raster.

    Returns:
    None
    """
    # Get a list of all files in the directory and then get a list of all .tifs
    files = os.listdir(rasters_path)
    tif_files = [file for file in files if file.endswith(".tif")]
    
    i = 0
    # Iterate through all the raster files in the input directory if ends with "tif"
    for tif_file in tif_files:
        i+=1
        print('splitting tile', i)
        img_id = tif_file.split(".tif")[0]
        raster = os.path.join(rasters_path, tif_file)
        # Open the input raster file using rasterio.open() and read the raster image and metadata information
        with rasterio.open(raster) as src:
            out_image = src.read()
            out_meta = src.meta
            out_transform = src.transform
            
            # Calculate the heights and widths of the tiles based on the specified width and height parameters
            heights = np.arange(0, out_image.shape[1], height)
            widths = np.arange(0, out_image.shape[2], width)
            
            # Extract the origin x (min) and y(max) of the raster image
            min_x = out_transform[2]
            max_y = out_transform[5]

            # Iterate through all the tiles and create a new tile image
            for x in range(len(heights)):
                for y in range(len(widths)):
                    # Skip the last tile if it is smaller than the specified height or width
                    if (x + 1 == len(heights)) or (y + 1 == len(widths)):
                        continue
                    else:
                        # Extract the new tile from the input raster using slice operation
                        new_tile = out_image[:, heights[x]:heights[x+1], widths[y]:widths[y+1]]
                        
                        # skip tiles if percentage black pixels is higher than 10%
                        if np.count_nonzero((new_tile[0] == 0) & (new_tile[1] == 0) & (new_tile[2] == 0)) > 0.1*new_tile[0].size:
                            continue
                        else:
                            # Calculate the transform for the new tile using rasterio.transform.from_origin() function
                            transform_new = rasterio.transform.from_origin(min_x + widths[y]/2, max_y - heights[x]/2, pixel_size, pixel_size)
                            
                            # Update the metadata information for the new tile
                            out_meta.update({"driver": "GTiff",
                                            "height": new_tile.shape[1],
                                            "width": new_tile.shape[2],
                                            "transform": transform_new})
                            
                            # Save the new tile to the output directory using rasterio.open() and dest.write() functions
                            with rasterio.open(os.path.join(output_dir, "{}_{}_{}_{}.tif".format(i, img_id, heights[x], widths[y])),
                                            "w", compress='lzw', **out_meta) as dest:
                                dest.write(new_tile)
        print('done splitting tile', i)


In [10]:
def split_single_geotiff(tif_file, rasters_path, output_dir, width, height, pixel_size):
    print('starting')
    img_id = tif_file.split(".tif")[0]
    raster = os.path.join(rasters_path, tif_file)
    # Open the input raster file using rasterio.open() and read the raster image and metadata information
    with rasterio.open(raster) as src:
        out_image = src.read()
        out_meta = src.meta
        out_transform = src.transform
        
        # Calculate the heights and widths of the tiles based on the specified width and height parameters
        heights = np.arange(0, out_image.shape[1], height)
        widths = np.arange(0, out_image.shape[2], width)
        
        # Extract the origin x (min) and y(max) of the raster image
        min_x = out_transform[2]
        max_y = out_transform[5]

        # Iterate through all the tiles and create a new tile image
        for x in range(len(heights)):
            for y in range(len(widths)):
                # Skip the last tile if it is smaller than the specified height or width
                if (x + 1 == len(heights)) or (y + 1 == len(widths)):
                    continue
                else:
                    new_tile = out_image[:, heights[x]:heights[x+1], widths[y]:widths[y+1]]
                    # skip tiles if percentage black pixels is higher than 10%
                    if np.count_nonzero((new_tile[0] == 0) & (new_tile[1] == 0) & (new_tile[2] == 0)) > 0.1*new_tile[0].size:
                        continue
                    else:
                        # Extract the new tile from the input raster using slice operation
                        
                        # Calculate the transform for the new tile using rasterio.transform.from_origin() function
                        transform_new = rasterio.transform.from_origin(min_x + widths[y]/2, max_y - heights[x]/2, pixel_size, pixel_size)
                        
                        # Update the metadata information for the new tile
                        out_meta.update({"driver": "GTiff",
                                        "height": new_tile.shape[1],
                                        "width": new_tile.shape[2],
                                        "transform": transform_new})
                        
                        # Save the new tile to the output directory using rasterio.open() and dest.write() functions
                        with rasterio.open(os.path.join(output_dir, "{}_{}_{}.tif".format(img_id, heights[x], widths[y])),
                                        "w", compress='lzw', **out_meta) as dest:
                            dest.write(new_tile)

    print('done splitting tile', img_id)

In [11]:
def reproject(df_ds, current_crs, approximate_crs):
    """Reproject a geodataframe of a shapefile from one CRS to another.

    Args:
        df_ds (DataFrame): A geodataframe of a shapefile layer.
        current_crs (str): The EPSG code of the current CRS of the shapefile.
        approximate_crs (str): The EPSG code of the target CRS.

    Returns:
        DataFrame: A geodataframe of a shapefile layer with the new coordinate reference system.
    """

    # Extract the geometries from the geodataframe
    geometries = df_ds['geometry']
    
    # Extract the coordinates of the geometries
    coords = shapely.get_coordinates(geometries)
    
    # Set up a coordinate transformer to transform from the current CRS to the target CRS
    transformer = pyproj.Transformer.from_crs(current_crs, approximate_crs, always_xy=True)
    
    # Transform the coordinates to the target CRS
    new_coords = transformer.transform(coords[:, 0], coords[:, 1])
    
    # Update the geometries in the original geodataframe with the transformed coordinates
    return shapely.set_coordinates(geometries.copy(), np.array(new_coords).T)


In [28]:
def split_annotations_to_geojson(df, small_tiles_folder, geojson_folder):
    """
    Split annotations data into the raster tiles and convert to the format of train-net.py: 
    Pygeos to GPD to geojson.

    Args:
    df (GeoDataFrame): Vector dataset containing features to split.
    tiles_path (str): Path to the directory where the raster tiles are stored
    geojson_folder (str): Path to the directory where the output geojson files will be saved
    
    Returns:
    None
    """
    # Create spatial index for faster query
    #spatial index are the bounding boxes of the asset geometries
    spatial_index = shapely.STRtree(df.geometry)
    id_obj = 0
    
    # Get a list of all files in the directory
    files = os.listdir(small_tiles_folder)

    # Filter for only the files that end with ".tif"
    tif_tiles = [file for file in files if file.endswith(".tif")]

    # Iterate through each raster tile
    for tile in tif_tiles:
        input_file = os.path.join(small_tiles_folder,tile)
        with rasterio.open(input_file) as src:
            out_image = src.read()
            out_meta = src.meta
        # geom is the bounding box of the raster tile
        geom = shapely.box(*src.bounds)
        
        # Query overlapping geometries from geom (tile) and the asset (from the spatial index)
        check_overlaps = spatial_index.query(geom, predicate='intersects')

        # Create new geojson file for each overlapping geometry
        if len(check_overlaps) > 0:   
            print('found overlaps')         
            get_matches = df.iloc[check_overlaps]
            get_exact_overlap = shapely.intersection(get_matches['geometry'].values, geom)
            df_objs = pd.DataFrame()
            for shapely_geom, included_r, area in zip(get_exact_overlap, get_matches['Included'], get_matches['area']): # adjusted so that other data is also passed
                id_obj += 1
                df_row = pd.DataFrame()
                included = 1 if included_r == 1.0 else 0
                df_row['properties'] = [{"id": id_obj,
                                         "included": included,
                                         "area": area,
                                         "building": "yes"}]
                #convert shapely geometry to geopandas geometry
                df_row['geometry'] = gpd.GeoSeries([shapely_geom])
                df_objs = pd.concat([df_objs, df_row], ignore_index=True) # maybe check for different method
            print(df_objs)
            gdf_obj = gpd.GeoDataFrame(df_objs, geometry='geometry', crs="epsg:28992")
            gdf_obj.set_geometry(col='geometry', inplace=True)
                
            img_id = tile.split(".tif")[0]
            gdf_obj.to_file(os.path.join(geojson_folder, "{}.geojson".format(img_id)), driver="GeoJSON")

# split_annotations_to_geojson(df, small_tiles_folder, json_folder)

In [ ]:
### This script processes the tiles parallel

width = 1000 #width of the new tiles
height = 1000 #height of the new tiles
nso_pix = 0.5 #pixel size of the tiles

#make output directory if does not exist
if not os.path.exists(small_tiles_folder):
    os.makedirs(small_tiles_folder)

# Get a list of all files in the directory
files = os.listdir(big_tiles_folder)
tif_files = [file for file in files if file.endswith(".tif")]

num_processes = multiprocessing.cpu_count() // 10
print('using', num_processes, 'CPUs')
pool = multiprocessing.Pool(processes=num_processes)

# Map the cutting function to each .tif file using multiple processes
args=[(tif_file, big_tiles_folder, small_tiles_folder, width, height, nso_pix) for tif_file in tif_files]
pool.starmap(split_single_geotiff, args)

pool.close()
pool.join()
pool.terminate()

In [3]:
### This script tests if all tiles are properly processed

files = os.listdir(big_tiles_folder)
names = [file.strip(".tif") for file in files if file.endswith(".tif")]

small_tiles = os.listdir(small_tiles_folder)
for name in names:
    contains = [small for small in small_tiles if name in small]
    print(f'The tif file named {name} has {len(contains)} small_tiles')

The tif file named 20220316_110548_SV1-04_SV_RD_8bit_RGB_50cm_Hoogkarspel has 599 small_tiles
The tif file named 20220724_110831_SV1-04_SV_RD_8bit_RGB_50cm_Oostvaardersplassen has 634 small_tiles
The tif file named 20220303_101830_SV1-01_SV_RD_8bit_RGB_50cm_Geldrop has 656 small_tiles
The tif file named 20221009_105653_SV1-04_SV_RD_8bit_RGB_50cm_Veldhoven has 631 small_tiles
The tif file named 20220719_105318_SV1-04_SV_RD_8bit_RGB_50cm_Heel has 624 small_tiles
The tif file named 20220813_105359_SV1-03_SV_RD_8bit_RGB_50cm_Hunsel has 627 small_tiles
The tif file named 20220322_111229_SV1-04_SV_RD_8bit_RGB_50cm_CapelleAanDenIJssel has 631 small_tiles
The tif file named 20221009_111423_SV1-03_SV_RD_8bit_RGB_50cm_Rhoon has 709 small_tiles
The tif file named 20220424_105500_SV1-04_SV_RD_8bit_RGB_50cm_Hattem has 645 small_tiles
The tif file named 20220424_105511_SV1-04_SV_RD_8bit_RGB_50cm_Beuningen has 642 small_tiles
The tif file named 20220623_103209_SV1-01_SV_RD_8bit_RGB_50cm_Amsterdam has

In [20]:
### Reproject annotation shapefile crs to the same crs as the raster tiles ###

annotations_crs = "epsg:4326"
raster_crs = "epsg:28992"
# read and reproject annotations
df = pd.DataFrame(gpd.read_file(annotation_path).copy())
df.geometry = reproject(df, annotations_crs, raster_crs)
print(len(df))
print(df.columns)
print('done')

229
Index(['annotator', 'origin', 'Included', 'area', 'geometry'], dtype='object')
done


In [29]:
### Spliting annotations shapefile into the raster tiles & save it to geojson ###

# Make folder for geojsons
if not os.path.exists(json_folder):
    os.makedirs(json_folder)

# Split annotations data into the new raster tiles of width X and height Y
split_annotations_to_geojson(df, small_tiles_folder, json_folder)
print('done')

found overlaps
                                          properties   
0  {'id': 1, 'included': 0, 'area': 2084, 'buildi...  \

                                            geometry  
0  POLYGON ((248494.325 607802.755, 248498.883 60...  
found overlaps
                                          properties   
0  {'id': 2, 'included': 0, 'area': 11619, 'build...  \

                                            geometry  
0  MULTIPOLYGON (((164932.410 384147.749, 164929....  
found overlaps
                                          properties   
0  {'id': 3, 'included': 0, 'area': 2235, 'buildi...  \

                                            geometry  
0  MULTIPOLYGON (((76888.446 433996.689, 76891.49...  
found overlaps
                                          properties   
0  {'id': 4, 'included': 0, 'area': 5973, 'buildi...  \

                                            geometry  
0  POLYGON ((182020.681 425892.307, 182020.118 42...  
found overlaps
                                 

In [ ]:
### This script tests if all tiles have overlaps

files = os.listdir(big_tiles_folder)
names = [file.strip(".tif") for file in files if file.endswith(".tif")]

geojsons = os.listdir(json_folder)
for name in names:
    name_geojsons = [geojson for geojson in geojsons if name in geojson]
    print(name, "has", len(name_geojsons), "geojsons")

In [ ]:
# # Pyhtonic version to check if the geojsons are correct:
# def split_annotations_to_geojson_pythonic (df, tiles_path, geojson_folder):
#     """
#     Split annotations data into the raster tiles and convert to the format of train-net.py: 
#     Pygeos to GPD to geojson.

#     Args:
#     df (GeoDataFrame): Vector dataset containing features to split.
#     tiles_path (str): Path to the directory where the raster tiles are stored
#     geojson_folder (str): Path to the directory where the output geojson files will be saved
    
#     Returns:
#     None
#     """
#     # Create spatial index for faster query
#     spatial_index = shapely.STRtree(df.geometry)
    
#     # Iterate through each raster tile
#     for tile in os.listdir(tiles_path):
#         if not tile.endswith('.tif'):
#             continue
        
#         input_file = os.path.join(tiles_path, tile)
#         with rasterio.open(input_file) as src:
#             geom = shapely.box(*src.bounds)
#             overlaps = spatial_index.query(geom, predicate='intersects')
        
#         # Create new geojson file for each overlapping geometry
#         if len(overlaps) > 0:
#             matches = df.iloc[overlaps]
#             exact_overlap = shapely.intersection(matches.geometry.values, geom)
            
#             objs = []
#             for polygon in exact_overlap:
#                 shapely_geom = shapely.to_shapely(polygon)
#                 objs.append({'properties': {'id': len(objs) + 1, 'building': 'yes'},
#                              'geometry': shapely_geom})
#             gdf = gpd.GeoDataFrame(objs, geometry='geometry', crs=src.crs)
#             img_id = tile.split(".tif")[0]
#             gdf.to_file(os.path.join(geojson_folder, f"{img_id}.geojson"), driver="GeoJSON")